# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [7]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests




# Import API key
from api_keys import geoapify_key

In [8]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,katsuren-haebaru,26.3369,127.8719,30.01,88,20,3.09,JP,1721068470
1,1,bodaybo,57.8506,114.1933,15.16,92,99,0.60,RU,1721068471
2,2,bredasdorp,-34.5322,20.0403,10.74,87,84,0.97,ZA,1721068472
3,3,victorica,-36.2151,-65.4359,13.99,28,0,9.19,AR,1721068474
4,4,new norfolk,-42.7826,147.0587,11.62,99,99,1.28,AU,1721068475


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [9]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng', 'Lat', geo=True, tiles='OSM', size='Humidity', color='City', alpha=0.6,
    hover_cols=['City', 'Humidity'], title='City Humidity Map'
)


# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [15]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df[
    (city_data_df['Max Temp'] >= 20) &
    (city_data_df['Max Temp'] <= 30) &
    (city_data_df['Humidity'] <= 70) &
    (city_data_df['Cloudiness'] <= 50)
]
# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
print("Sample data after filtering:")
ideal_cities = pd.DataFrame(ideal_cities_df)

ideal_cities.head()

Sample data after filtering:


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,10,lihue,21.9789,-159.3672,25.97,70,40,8.23,US,1721068482
44,44,balkanabat,39.5108,54.3671,27.41,52,0,3.33,TM,1721068528
69,69,colonia,50.9333,6.9500,25.70,63,0,2.06,DE,1721068563
97,97,avarua,-21.2078,-159.7750,22.03,60,40,3.09,CK,1721068604
107,107,yuzhno-sukhokumsk,44.6581,45.6436,29.64,32,5,4.63,RU,1721068616


### Step 3: Create a new DataFrame called `hotel_df`.

In [17]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
10,lihue,US,21.9789,-159.3672,70,
44,balkanabat,TM,39.5108,54.3671,52,
69,colonia,DE,50.9333,6.9500,63,
97,avarua,CK,-21.2078,-159.7750,60,
107,yuzhno-sukhokumsk,RU,44.6581,45.6436,32,
112,yellowknife,CA,62.4560,-114.3525,59,
121,mancio lima,BR,-7.6142,-72.8958,61,
127,tamanrasset,DZ,22.7850,5.5228,57,
133,alghero,IT,40.5589,8.3181,61,
141,uvinza,TZ,-5.1036,30.3911,45,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [18]:
# Set parameters to search for a hotel
radius = 10000

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params = {
        "categories": "accommodation.hotel",
        "filter": f"circle:{longitude},{latitude},{radius}",
        "bias": f"proximity:{longitude},{latitude}",
        "apiKey": geoapify_key,
        "limit": 1
    }

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found"
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
lihue - nearest hotel: Kauai Palms
balkanabat - nearest hotel: Nebitçi
colonia - nearest hotel: Wasserturm Hotel Cologne
avarua - nearest hotel: Paradise Inn
yuzhno-sukhokumsk - nearest hotel: No hotel found
yellowknife - nearest hotel: No hotel found
mancio lima - nearest hotel: No hotel found
tamanrasset - nearest hotel: فندق الأمان
alghero - nearest hotel: Catalunya
uvinza - nearest hotel: Mapito lodge
newman - nearest hotel: No hotel found
tutoia - nearest hotel: Pousada San Vicente
davydovka - nearest hotel: No hotel found
luau - nearest hotel: No hotel found
kapa'a - nearest hotel: Pono Kai Resort
birjand - nearest hotel: مهمانسرای جهانگردی بیرجند
diamantino - nearest hotel: Hotel kayaby
altay - nearest hotel: Sunshine Holiday Hotel
nitra - nearest hotel: Grand Hotel Sole
lata - nearest hotel: Residencial Botânico
inderbor - nearest hotel: No hotel found
vila velha - nearest hotel: Hotel Prainha
patos de minas - nearest hotel: Gálatas Central Hotel
coeur d'a

,City,Country,Lat,Lng,Humidity,Hotel Name
10,lihue,US,21.9789,-159.3672,70,Kauai Palms
44,balkanabat,TM,39.5108,54.3671,52,Nebitçi
69,colonia,DE,50.9333,6.9500,63,Wasserturm Hotel Cologne
97,avarua,CK,-21.2078,-159.7750,60,Paradise Inn
107,yuzhno-sukhokumsk,RU,44.6581,45.6436,32,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [19]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    'Lng', 'Lat', geo=True, tiles='OSM', size='Humidity', color='City', alpha=0.6,
    hover_cols=['City', 'Country', 'Hotel Name', 'Humidity'], title='City Humidity and Hotel Map'
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)